# Evaluating Accuracies of Probabilistic Quantile Regression and Classification Models

### Install Dependencies

In [ ]:
!pip install scikit-learn==1.5.2
# Install TabPFN
!pip install tabpfn
# Also install random forest quantile regression
!pip install quantile-forest
# install sigmaeval
!pip install sigmaeval

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import r2_score, roc_auc_score
from quantile_forest import RandomForestQuantileRegressor

from tabpfn import TabPFNRegressor, TabPFNClassifier

from sigmaeval.sigmaeval import crps_quantile, brier_score_multiclass

## Test Regression Quantile Accuracies

As evaluation of the accuary of the predictive posterior (heregiven by quantiles) we use the **Continuous Ranked Probability Score (CRPS)**.

CRPS  is a proper scoring rule used to assess the accuracy of probabilistic predictions, particularly for continuous outcomes. It measures how well a predicted cumulative distribution function (CDF) aligns with the observed value. CRPS generalizes the **Brier Score** to continuous variables and accounts for both **calibration** (how well predicted probabilities match actual frequencies) and **sharpness** (how concentrated the predictive distribution is).


### **Advantages of CRPS for Quantile-Based Predictions**

✅ **Handles Uncertainty Properly:** Unlike MSE, which only considers point predictions, CRPS evaluates the full distribution.  
✅ **Comparable Across Different Models:** Since CRPS measures the entire distribution’s performance, it is useful for comparing probabilistic models.  
✅ **Works with Prediction Intervals:** Even if the model only predicts quantiles, CRPS can approximate how well those quantiles match the actual distribution.  

### Load Dataset and Fit Models

Here we compare TabPFN to RandomForests quantile regression.

In [6]:
X, y = datasets.fetch_california_housing(return_X_y=True)
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 500, test_size=500, random_state=42)

In [7]:
model_pfn = TabPFNRegressor()
model_pfn.fit(X_train, y_train)

TabPFNRegressor()

In [8]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
pred_quantiles_pfn = model_pfn.predict(X_test, output_type="quantiles", quantiles=quantiles)
pred_quantiles_pfn = np.asarray(pred_quantiles_pfn)

In [9]:
model_rf = RandomForestQuantileRegressor()
model_rf.fit(X_train, y_train)
# get predictions and quantiles
pred_rf = model_rf.predict(X_test, quantiles=quantiles)

In [13]:
# Evaluate R2 score for the median prediction accuracy of the two models
r2_pfn = r2_score(y_test, pred_quantiles_pfn[4, :])
r2_rf = r2_score(y_test, pred_rf[:, 4])
print(f"R2 score for TabPFN: {round(r2_pfn,4)}")
print(f"R2 score for RandomForest: {round(r2_rf,4)}")

R2 score for TabPFN: 0.7946
R2 score for RandomForest: 0.7099


In [14]:
# Evaluate CRPS for the two models
crps_pfn = crps_quantile(pred_quantiles_pfn, quantiles, y_test)
print(f"CRPS TabPFB: {round(crps_pfn,4)}")
crps_rf = crps_quantile(pred_rf.T, quantiles, y_test)
print(f"CRPS RandomForest: {round(crps_rf,4)}")

CRPS TabPFB: 0.1626
CRPS RandomForest: 0.2178


## Test Classification Probabilities

The **Brier Score** is a proper scoring rule that measures the accuracy of probabilistic predictions. For **multi-class classification**, the generalized **multi-class Brier Score** is defined as:

$$
\text{Brier Score} = \frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^{K} (p_{ik} - y_{ik})^2
$$

where:
* $N$ is the number of samples
* $K$ is the number of classes
* $p_{ik}$ is the predicted probability for class $k$ in sample $i$
* $y_{ik}$ is a one-hot encoded ground-truth indicator (1 if true class, 0 otherwise)

## Key Properties

* **Proper scoring rule**: Encourages well-calibrated probabilistic predictions
* **Lower values are better**: A perfect prediction gives a Brier score of 0
* **Works for multi-class settings**: Unlike log-loss, it doesn't require log-probabilities


### Test on Multi-Class Data

In [30]:
data = datasets.fetch_openml(name="parkinsons")
print(data.DESCR)

**Author**:   
**Source**: UCI
**Please cite**: 'Exploiting Nonlinear Recurrence and Fractal Scaling Properties for Voice Disorder Detection', Little MA, McSharry PE, Roberts SJ, Costello DAE, Moroz IM. BioMedical Engineering OnLine 2007, 6:23 (26 June 2007) 

* Abstract: 

Oxford Parkinson's Disease Detection Dataset

* Source:

The dataset was created by Max Little of the University of Oxford, in collaboration with the National Centre for Voice and Speech, Denver, Colorado, who recorded the speech signals. The original study published the feature extraction methods for general voice disorders.

* Data Set Information:
This dataset is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the table is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column). The main aim of the data is to discriminate healthy people from those with PD, according to "status" colu

In [37]:
X = data.data
y = data.target
categories = data.categories
feature_names = data.feature_names
print('X shape:', X.shape)

X shape: (195, 22)


In [38]:
# split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [39]:
# TabPFN Classifier
model_pfn = TabPFNClassifier()
model_pfn.fit(X_train, y_train)

TabPFNClassifier()

In [60]:
pred_pfn = model_pfn.predict_proba(X_test)
brier_pnf = brier_score_multiclass(y_test.to_numpy().astype(int)-1,pred_pfn)
print(f"Brier Score TabPFN: {round(float(brier_pnf),4)}")

Brier Score TabPFN: 0.0592


In [57]:
# Compare with RandomForest
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict_proba(X_test)
brier_rf = brier_score_multiclass(y_test.to_numpy().astype(int)-1,pred_rf)
print(f"Brier Score RandomForest: {round(brier_rf,4)}")

Brier Score RandomForest: 0.0674
